In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

Ознайомитись з описом датасета, визначити цільову змінну (target) та основну задачу прогнозної моделі. 

In [ ]:
data = pd.read_csv('/kaggle/input/telecon/bigml_59c28831336c6604c800002a.csv')

In [ ]:
data.info()

In [ ]:
data.head()

Цільова змінна - *churn*
Основна задача прогрозної моделі - спрогнозувати, чи відмовиться клієнт від послуг телефоної компанії

Виконати попередню обробку даних з метою подальшої побудови моделі.

In [ ]:
encoded_cat, _ = pd.factorize(data['voice mail plan'])
print(stats.pointbiserialr(data['number vmail messages'], encoded_cat))

In [ ]:
c_tab = pd.crosstab(data['voice mail plan'], data['churn'])
corr = stats.contingency.association(c_tab)
print("Correlation —", corr)
p_value = stats.contingency.chi2_contingency(c_tab).pvalue
print("P_value —", p_value)

In [ ]:
c_tab = pd.crosstab(data['state'], data['churn'])
corr = stats.contingency.association(c_tab)
print("Correlation —", corr)
p_value = stats.contingency.chi2_contingency(c_tab).pvalue
print("P_value —", p_value)

In [ ]:
print(stats.pointbiserialr(data['churn'], data['customer service calls']))

In [ ]:
c_tab = pd.crosstab(data['international plan'], data['churn'])
corr = stats.contingency.association(c_tab)
print("Correlation —", corr)
p_value = stats.contingency.chi2_contingency(c_tab).pvalue
print("P_value —", p_value)

В першу чергу, видалимо змінні, що є або лінійною комбінацією інших, або мають статистичну незначущість зі змінною *churn* 

In [ ]:
columns_to_drop = ['area code', 'phone number', 'total day charge', 'total eve charge', 
                   'total night charge', 'total intl charge', 'number vmail messages']
data = data.drop(columns_to_drop, axis = 1)

In [ ]:
data.head()

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(10,6))
i_c = ['total day minutes', 'total day calls', 'total eve minutes', 'total eve calls', 
       'total night calls', 'total intl minutes', 'account length', 'total night minutes']
for j in range(2):
    for i in range(4):
        sns.histplot(ax=axes[j, i], data = data, x=  i_c[i+j])
        print(stats.skew(data[i_c[i+j]]), stats.kurtosis(data[i_c[i+j]]))

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

oe = OrdinalEncoder()
ss = StandardScaler()
mms = MinMaxScaler()

bin_cat = ['international plan', 'voice mail plan']


X = data.drop(columns = ['churn'])
y = oe.fit_transform(data['churn'].to_numpy().reshape(-1, 1)).ravel()

X[i_c] = ss.fit_transform(X[i_c])
X[bin_cat] = oe.fit_transform(X[bin_cat])
#X['total intl calls'] = mms.fit_transform(X['total intl calls'].to_numpy().reshape(-1, 1))

filt = X['customer service calls'] < 4
X[filt]['customer service calls'] = 0
X[~filt]['customer service calls'] = X[~filt]['customer service calls'] - 3 


X = pd.get_dummies(X, columns=['state'])

Побудувати одну з лінійних моделей машинного навчання (лінійну регресію або логістичну регресію, залежно від вашого варіанту). Оцінити якість моделі на тестових даних за допомогою декількох метрик. Зробити висновки

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score 
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=4)
lr = LogisticRegression(penalty = None)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_valid)
print(f"Accuracy -- {accuracy_score(y_pred, y_valid)}")
print(confusion_matrix(y_valid, y_pred))

In [ ]:
metrics = ['accuracy', 'roc_auc', 'f1', 'precision','recall']

In [ ]:
lr = LogisticRegression(penalty = None)
cross_valid_score_list = cross_validate(lr, X, y,
                                        cv=5, scoring = metrics)
c_v = pd.DataFrame(cross_valid_score_list)[['test_accuracy', 'test_roc_auc', 'test_f1',   'test_precision', 'test_recall']].mean()
print(c_v)

Як ми бачимо, точність самої моделі - добра. Але це викликано дизбалансом target-класу.
Так, якщо подивитися на precision та recall, ми побачимо гірші результати. Precision нам говорить, що модель кожен другий sample відносить до правильного класу, тоді як recall - 4 з 5 семплів - помилково відносить до іншого класу

Застосувати декілька типів регуляризації (Ridge, Lasso, ElasticNet), налаштувати гіперпараметри моделей, побудувати валідаційні криві. Оцінити якість отриманих моделей, порівняти між собою. Зробити висновки. (5 балів)

In [ ]:
lr_l1 = LogisticRegression(penalty = 'l1', solver = 'saga', max_iter = 1333)
lr_l2 = LogisticRegression(max_iter = 1333)
lr_el = LogisticRegression(penalty = 'elasticnet', solver = 'saga', l1_ratio = 0.5, max_iter = 1333)

lr = [['L1', lr_l1], ['L2', lr_l2], ['ElasticNet', lr_el]]
for model in lr:
    cross_valid_score_list = cross_validate(model[1], X, y,
                                            cv=5, scoring = metrics)
    c_v = pd.DataFrame(cross_valid_score_list)[['test_accuracy', 'test_roc_auc', 'test_f1',   'test_precision', 'test_recall']].mean()
    print(model[0], end = ':')
    print(c_v)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=4)

X_resampled, y_resampled = rus.fit_resample(X, y)

In [ ]:
for model in lr:
    cross_valid_score_list = cross_validate(model[1], X_resampled,y_resampled,
                                            cv=5, scoring = metrics)
    c_v = pd.DataFrame(cross_valid_score_list)[['test_accuracy', 'test_roc_auc', 'test_f1',   'test_precision', 'test_recall']].mean()
    print(model[0], end = ':')
    print(c_v)

Кращий precision та f1 - score зі l1-регулярізацією. Надалі будемо її використовувати

In [ ]:
from sklearn.model_selection import GridSearchCV 

scores = []
for model in lr:
    RS_CV = GridSearchCV(model[1], {'C': np.logspace(-1, 10, 20)})
    RS_CV.fit(X_resampled, y_resampled)
    scores.append(RS_CV)
    print(f"{model[0]} -- Best parameters are {RS_CV.best_params_}, and score -- {RS_CV.best_score_}")

In [ ]:
for i in range(3):
    results_df = pd.DataFrame(scores[i].cv_results_)
    plt.plot(results_df['param_C'], results_df['mean_test_score'], label=lr[i][0])

# Подписываем оси и график
plt.xlabel('C')
plt.ylabel('Test accuracy')
plt.title('Validation curve')
plt.legend()
plt.show()

**Висновки** Моделі справляються із задачею однаково добре, незважаючи на невелику похибку: accuracy, precision та recall мають дуже маленьке средньоквадратичне відхилення

Вивести (або візуалізувати) коефіцієнти найкращої з отриманих моделей, зробити висновок щодо впливовості факторів на результуючу змінну (target). (2 бали)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=4)
lr = LogisticRegression(penalty = 'l1', solver = 'saga', C = 0.52, max_iter = 1500)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_valid)

In [ ]:
feauture, importance = np.reshape(lr.feature_names_in_, 63), np.reshape(np.round(RS_CV.best_estimator_.coef_, 4), 63)

num_to_plot = 12

indices = np.argsort(importance)[::-1]

for i in range(num_to_plot):
    print(i+1,feauture[indices[i]], importance[indices[i]],)

plt.figure(figsize=(17,5))
plt.title("Feature importances")
bars = plt.bar(range(num_to_plot),
               importance[indices[:num_to_plot]],
               color=([str(i/float(num_to_plot+1)) for i in range(num_to_plot)]),
               align="center")

ticks = plt.xticks(range(num_to_plot),
                   feauture[indices[:num_to_plot]])

plt.xlim([-1, num_to_plot])
plt.legend(bars, [u''.join(feauture[i]) for i in indices[:num_to_plot]]);

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state = 4).fit(X_train, y_train)

y_pred = rfc.predict(X_valid)
print(f"Accuracy -- {round(accuracy_score(y_pred, y_valid),3)*100}%")
print(f"Precision -- {round(precision_score(y_pred, y_valid),3)*100}%")
print(f"Recall -- {round(recall_score(y_pred, y_valid),3)*100}%")
print(f"F1 -- {round(f1_score(y_pred, y_valid),3)*100}%")



In [ ]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {'n_estimators': np.arange(100, 300, 20, dtype = int),
              'max_depth': np.arange(5, 15, dtype = int),
              'min_samples_split':np.arange(2, 9, dtype = int),
              'min_samples_leaf': np.arange(1, 8, dtype = int),
              'max_features': np.arange(4, 10, dtype = int)
              }

rfc = RandomForestClassifier(random_state = 4)

clf = RandomizedSearchCV(rfc, parameters, random_state=0, cv = 10)

clf.fit(X_resampled, y_resampled)



In [ ]:
print(f"Best parameters are {clf.best_params_}")

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=4)

rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_valid)
print(f"Accuracy -- {round(accuracy_score(y_pred, y_valid),3)*100}%")
print(f"Precision -- {round(precision_score(y_pred, y_valid),3)*100}%")
print(f"Recall -- {round(recall_score(y_pred, y_valid),3)*100}%")
print(f"F1 -- {round(f1_score(y_pred, y_valid),3)*100}%")

In [ ]:
rfc_1 = RandomForestClassifier(min_samples_split = 8, min_samples_leaf = 3,
                             random_state = 12, n_estimators = 260)

cross_valid_score_list = cross_validate(rfc_1, X, y,
                                            cv=5, scoring = metrics)
c_v = pd.DataFrame(cross_valid_score_list)[['test_accuracy', 'test_roc_auc', 'test_f1',   'test_precision', 'test_recall']].mean()
print("Random Forest with unbalanced data")
print(c_v)

print("Random Forest with undersampled data")
cross_valid_score_list = cross_validate(rfc_1, X_resampled, y_resampled,
                                            cv=5, scoring = metrics)
c_v = pd.DataFrame(cross_valid_score_list)[['test_accuracy', 'test_roc_auc', 'test_f1',   'test_precision', 'test_recall']].mean()
print(c_v)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 5, p = 4)

cross_valid_score_list = cross_validate(knn, X, y,
                                            cv=5, scoring = metrics)
c_v = pd.DataFrame(cross_valid_score_list)[['test_accuracy', 'test_roc_auc', 'test_f1',   'test_precision', 'test_recall']].mean()
print("KNN with unbalanced data")
print(c_v)

print("KNN with undersampled data")
cross_valid_score_list = cross_validate(knn, X_resampled, y_resampled,
                                            cv=5, scoring = metrics)
c_v = pd.DataFrame(cross_valid_score_list)[['test_accuracy', 'test_roc_auc', 'test_f1',   'test_precision', 'test_recall']].mean()
print(c_v)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(min_samples_split = 5, min_samples_leaf = 2, max_features =  9, max_depth = 13)

cross_valid_score_list = cross_validate(dtc, X, y,
                                            cv=5, scoring = metrics)
c_v = pd.DataFrame(cross_valid_score_list)[['test_accuracy', 'test_roc_auc', 'test_f1',   'test_precision', 'test_recall']].mean()
print("KNN with unbalanced data")
print(c_v)

print("KNN with undersampled data")
cross_valid_score_list = cross_validate(dtc, X_resampled, y_resampled,
                                            cv=5, scoring = metrics)
c_v = pd.DataFrame(cross_valid_score_list)[['test_accuracy', 'test_roc_auc', 'test_f1',   'test_precision', 'test_recall']].mean()
print(c_v)